### if you already have cpptraj input file and don't want to retype (loading traj, specify command, ...)? Use "load_cpptraj_file"

In [1]:
# test folder can be found here: https://github.com/mojyt/cpptraj/tree/master/test/Test_Comprehensive
# I just add "parm" file to `ptraj.in` and updat relative path
# TODO : more detail
!cat ptraj.in

parm ../cpptraj/test/Test_Comprehensive/compound.prmtop
# system: 7mer peptide in a box of
# periodic water (1577 Waters)

# Read in the trajectory file 
# starting at 1
# 100 total snapshots
# consider each one 

trajin ../cpptraj/test/Test_Comprehensive/trajectory.mdcrd 1 100 1

#  first calculate some things that don't depend on
#  the position of the solute in the box

# do dihedral angles

dihedral phi :1@C  :2@N  :2@CA :2@C out phi
dihedral psi   :1@N  :1@CA :1@C  :2@N out psi
dihedral omega   :1@CA :1@C  :2@N  :2@CA out omega

# calculate the distance between two atoms 
# over the 10 snapshots
# output to the file "dist.list"

distance end_to_end :1@N :7@N out  dist_end_to_end.list

#get the rms values for the whole system 
#referenced to the first snapshot

rms first :1-1584

#now deal with drift

# the peptide (residues 1->7) has drifted around 
# the box during MD.  We'd like to view the system
# with a "constant" solute.

# note that this changes the coordinates you have in 

## lazy loading

In [2]:
from __future__ import print_function # for python 2 and 3 compat
import pytraj as pt

# load cpptraj file and return a `state`, which hold lots of information (TopologyList, DataSetList, TrajinList, ...).
# you can take the raw data from this `state`

In [3]:
# load state from `ptraj.in`
state = pt.load_cpptraj_file("ptraj.in")
state.run()

CpptrajState, include:
<datasetlist: 7 datasets>

In [4]:
# we did save traj to disk
!ls test.mdcrd

test.mdcrd


In [5]:
print(state.data.keys())

['phi', 'psi', 'omega', 'end_to_end', 'RMSD_00004', 'ws[lower]', 'ws[upper]']


In [6]:
print(state.data['phi'])

<pytraj.datasets.DatasetDouble: size=100, key=phi> 
values: 
[ 41.28882154  51.91785129  49.05564944 ...,  51.2927548   61.26562317
  53.001836  ]
